# Importing the relevant libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings, string
warnings.filterwarnings('ignore')
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
import gensim.downloader as api
from nltk import word_tokenize
import joblib
from sklearn.pipeline import Pipeline

## Loading the dataset

In [2]:
df = pd.read_csv('Precily_Text_Similarity.csv')
df.head()

,text1,text2
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...
2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...
3,hearts of oak 3-2 cotonsport hearts of oak set...,redford s vision of sundance despite sporting ...
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...


# Text Cleaning and Preprocessing

In [3]:
stemmer = PorterStemmer()
def stem_words(text):
    return ' '.join([stemmer.stem(word) for word in text.split()])
df['text1'] = df['text1'].apply(lambda x: stem_words(x))
df['text2'] = df['text2'].apply(lambda x: stem_words(x))

In [4]:
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
df['text1'] = df['text1'].apply(lambda x: lemmatize_words(x))
df['text2'] = df['text2'].apply(lambda x: lemmatize_words(x))

In [5]:
def text_process(text):
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return ' '.join([word for word in nopunc.split() if word.lower() not in stopwords.words('english') and not word.isdigit()])

In [6]:
df.shape

(3000, 2)

In [7]:
df['text1'] = df['text1'].apply(lambda x: text_process(x))
df['text2'] = df['text2'].apply(lambda x: text_process(x))

In [8]:
saved_df = df.copy()

## Jaccard Similarity Technique

The Jaccard similarity index measures the similarity between two sets of data. It can range from 0 to 1. The higher the number, the more similar the two sets of data.

The Jaccard similarity index is calculated as:

Jaccard Similarity = (number of observations in both sets) / (number in either set)

Or, written in notation form:

J(A, B) = |A∩B| / |A∪B|

In [9]:
def jaccard_similarity(a,b):
    intersection = set(a).intersection(set(b))
    union = set(a).union(set(b))
    return len(intersection)/len(union)

In [10]:
jaccard_similarity(df['text1'][0],df['text2'][0])

0.7222222222222222

# Text Vectorization 

### Count Vectorizer - TF(Term Frequency)

We'll convert each message, represented as a list of tokens, into a vector that machine learning models can understand.

We'll do that in three steps using the bag-of-words model:

1. Count how many times does a word occur in each message (Known as term frequency)

2. Weigh the counts, so that frequent tokens get lower weight (inverse document frequency)

3. Normalize the vectors to unit length, to abstract from the original text length (L2 norm)


Each vector will have as many dimensions as there are unique words in the SMS corpus.  We will first use SciKit Learn's **CountVectorizer**. This model will convert a collection of text documents to a matrix of token counts.

We can imagine this as a 2-Dimensional matrix. Where the 1-dimension is the entire vocabulary (1 row per word) and the other dimension are the actual documents, in this case a column per text message. 

In [11]:
bow_transformer1 = CountVectorizer(analyzer=text_process).fit(df['text1'])
bow_transformer2 = CountVectorizer(analyzer=text_process).fit(df['text2'])
print(bow_transformer1)
print(bow_transformer2)

CountVectorizer(analyzer=<function text_process at 0x000002798A903280>)
CountVectorizer(analyzer=<function text_process at 0x000002798A903280>)


In [12]:
len(bow_transformer1.vocabulary_), len(bow_transformer2.vocabulary_)

(38, 38)

Let's consider the 4th message of the text1 column of the given dataset and generate bag of words along with their frequency.

In [13]:
text4 = df['text1'][3]
bow4 = bow_transformer1.transform([text4])
print(bow4)

  (0, 0)	167
  (0, 4)	1
  (0, 6)	1
  (0, 9)	1
  (0, 10)	1
  (0, 11)	95
  (0, 12)	17
  (0, 13)	32
  (0, 14)	23
  (0, 15)	88
  (0, 16)	24
  (0, 17)	22
  (0, 18)	23
  (0, 19)	56
  (0, 20)	3
  (0, 21)	19
  (0, 22)	39
  (0, 23)	31
  (0, 24)	55
  (0, 25)	70
  (0, 26)	12
  (0, 27)	4
  (0, 28)	55
  (0, 29)	35
  (0, 30)	72
  (0, 31)	29
  (0, 32)	3
  (0, 33)	13
  (0, 34)	1
  (0, 35)	6
  (0, 36)	2


In [14]:
bow_text1 = bow_transformer1.transform(df['text1'])
print(bow_text1.shape)

(3000, 38)


In [15]:
bow_text2 = bow_transformer2.transform(df['text2'])
print(bow_text2.shape)

(3000, 38)


## TF-IDF Transformer(Term Frequency-Inverse Document Frequency)

TF-IDF stands for *term frequency-inverse document frequency*, and the tf-idf weight is a weight often used in information retrieval and text mining. This weight is a statistical measure used to evaluate how important a word is to a document in a collection or corpus. The importance increases proportionally to the number of times a word appears in the document but is offset by the frequency of the word in the corpus. 

Typically, the tf-idf weight is composed by two terms: the first computes the normalized Term Frequency (TF), aka. the number of times a word appears in a document, divided by the total number of words in that document; the second term is the Inverse Document Frequency (IDF), computed as the logarithm of the number of the documents in the corpus divided by the number of documents where the specific term appears.

**TF: Term Frequency**, which measures how frequently a term occurs in a document. Since every document is different in length, it is possible that a term would appear much more times in long documents than shorter ones. Thus, the term frequency is often divided by the document length (aka. the total number of terms in the document) as a way of normalization: 

*TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).*

**IDF: Inverse Document Frequency**, which measures how important a term is. While computing TF, all terms are considered equally important. However it is known that certain terms, such as "is", "of", and "that", may appear a lot of times but have little importance. Thus we need to weigh down the frequent terms while scale up the rare ones, by computing the following: 

*IDF(t) = log_e(Total number of documents / Number of documents with term t in it).*

#### Using Sklearn's TFIDF Transformer to transform the entire bag of words corpus into TFIDF corpus:

In [16]:
tfidf_transformer1 = TfidfTransformer().fit(bow_text1)
tfidf_transformer2 = TfidfTransformer().fit(bow_text2)

In [17]:
tfidf4 = tfidf_transformer1.transform(bow4)
print(tfidf4)

  (0, 36)	0.013055846780139978
  (0, 35)	0.022250077435161244
  (0, 34)	0.004138953140290266
  (0, 33)	0.04820850110951602
  (0, 32)	0.011128746445899422
  (0, 31)	0.10754204093661267
  (0, 30)	0.2670009292219349
  (0, 29)	0.1297921183717739
  (0, 28)	0.20395904315564473
  (0, 27)	0.0214428335884421
  (0, 26)	0.04450015487032249
  (0, 25)	0.2595842367435478
  (0, 24)	0.20395904315564473
  (0, 23)	0.11495873341499975
  (0, 22)	0.14462550332854807
  (0, 21)	0.07088246164593452
  (0, 20)	0.012989252523926654
  (0, 19)	0.20766738939483825
  (0, 18)	0.08529196350145143
  (0, 17)	0.0815836172622579
  (0, 16)	0.08900030974064498
  (0, 15)	0.3263344690490316
  (0, 14)	0.08529196350145143
  (0, 13)	0.11866707965419329
  (0, 12)	0.06304188606629019
  (0, 11)	0.3522928927233863
  (0, 10)	0.011089971430165385
  (0, 9)	0.010339242840822743
  (0, 6)	0.008505999327078297
  (0, 4)	0.009006977779888707
  (0, 0)	0.6192938219453212


In [18]:
tfidf4.get_shape()

(1, 38)

#### To transform the entire bag of words corpus into TFIDF corpus at once:

In [19]:
tfidf_text1 = tfidf_transformer1.transform(bow_text1)
tfidf_text2 = tfidf_transformer2.transform(bow_text2)

### Checking out the various features of the TFIDF sparse matrices of the 2 text columns:

In [20]:
print("Amount of non-zero values in TFIDF of text 1:",tfidf_text1.nnz)
print("Amount of non-zero values in TFIDF of text 2:",tfidf_text2.nnz)

Amount of non-zero values in TFIDF of text 1: 85549
Amount of non-zero values in TFIDF of text 2: 85711


In [21]:
print("Shape of TFIDF of text 1:",tfidf_text1.shape)
print("Shape of TFIDF of text 2:",tfidf_text2.shape)

Shape of TFIDF of text 1: (3000, 38)
Shape of TFIDF of text 2: (3000, 38)


In [22]:
print("Sparsity of text 1:",str(np.round((tfidf_text1.nnz/(tfidf_text1.shape[0]*tfidf_text1.shape[1]))*100,2)) + '%')
print("Sparsity of text 2:",str(np.round((tfidf_text2.nnz/(tfidf_text2.shape[0]*tfidf_text2.shape[1]))*100,2)) + '%')

Sparsity of text 1: 75.04%
Sparsity of text 2: 75.19%


## Cosine Similarity

Cosine similarity is a metric used to measure how similar the documents are irrespective of their size. Mathematically, it measures the cosine of the angle between two vectors projected in a multi-dimensional space.

The cosine similarity is advantageous because even if the two similar documents are far apart by the Euclidean distance (due to the size of the document), chances are they may still be oriented closer together. The smaller the angle, higher the cosine similarity.  

### Evaluating the cosine similarity between any two TFIDF vectors in order to determine the degree of lexical and semantic resemblance, closeness or similarity of the two vectors...

In [23]:
cos_similarity = [[]]
for i in range(tfidf_text1.shape[0]):
    cos_similarity.append(cosine_similarity(tfidf_text1[i],tfidf_text2))
cos_similarity = pd.DataFrame(cos_similarity)
cos_similarity.drop(index=cos_similarity.index[0], 
        axis=0, 
        inplace=True)
cos_similarity.head()

,0
1,"[0.9785293861463167, 0.9887675564206015, 0.988..."
2,"[0.970073371768313, 0.9818744094402776, 0.9865..."
3,"[0.9643083804078563, 0.9701381455232274, 0.962..."
4,"[0.9759967531076594, 0.982251639784895, 0.9806..."
5,"[0.9834646778322373, 0.9882872871600276, 0.994..."


In [24]:
cos_similarity.shape

(3000, 1)

## Generating the cosine similarity matrix consisting of the cosine similarity scores of each and every text present in both the columns 'text1' and 'text2'
### It is of dimensions 3000X3000 as each text paragraph in 'text1' column is mapped with each of the 3000 text paragraphs of the 'text2' column.

In [25]:
cosine_similarity_matrix = pd.DataFrame()
for i in range(cos_similarity.shape[0]):
    cosine_similarity_matrix = pd.concat([cosine_similarity_matrix,(pd.DataFrame(cos_similarity.iloc[i].values.tolist()).T)],axis=1)
cosine_similarity_matrix.head()

,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0.978529,0.970073,0.964308,0.975997,0.983465,0.974899,0.975446,0.981171,0.974635,0.971818,...,0.976139,0.977350,0.982670,0.970279,0.969073,0.979111,0.982001,0.961637,0.979260,0.975768
1,0.988768,0.981874,0.970138,0.982252,0.988287,0.981370,0.978910,0.989982,0.980058,0.979575,...,0.981408,0.985446,0.983493,0.978564,0.971599,0.977041,0.990116,0.969929,0.981814,0.987708
2,0.988882,0.986596,0.962266,0.980612,0.994628,0.977163,0.977388,0.986011,0.980424,0.985989,...,0.982220,0.980308,0.975918,0.981216,0.977847,0.976986,0.986111,0.960132,0.980092,0.977371
3,0.984609,0.973072,0.960169,0.976659,0.985454,0.973015,0.982949,0.994004,0.984009,0.982400,...,0.988786,0.983359,0.991103,0.984215,0.978290,0.985069,0.983268,0.969183,0.984444,0.984504
4,0.991294,0.981296,0.956340,0.986461,0.989491,0.980954,0.970051,0.984327,0.974304,0.979598,...,0.974879,0.974234,0.981270,0.971890,0.969613,0.974051,0.984597,0.962306,0.977571,0.974417


In [26]:
cosine_similarity_matrix.shape

(3000, 3000)

In [27]:
cosine_similarity_matrix.idxmin()

0    707
0    707
0    707
0    614
0    707
    ... 
0    586
0    707
0    614
0    707
0    707
Length: 3000, dtype: int64

In [28]:
cosine_similarity_matrix.iloc[707].min()

0.881484386605357

## Leveraging Google News Word 2 Vector library present in gensim.downloader module to generate a huge inbuilt word vocabulary which can be used to check for semantic similarity between the two texts:

In [29]:
wv = api.load('word2vec-google-news-300')

## Function to calculate the semantic similarity between the two text documents based on the concept of word embeddings:

In [30]:
similarity = []

for idx in df.index:
    t1 = df['text1'][idx]
    t2 = df['text2'][idx]
    
    if t1 == t2:
        similarity.append(1)
    else:
        t1_words = word_tokenize(t1)
        t2_words = word_tokenize(t2)
        vocab = wv.vocab
        
        if len(t1_words and t2_words) == 0:
            similarity.append(0)
        else:
            for word in t1_words.copy():
                if word not in vocab:
                    t1_words.remove(word)
            for word in t2_words.copy():
                if word not in vocab:
                    t2_words.remove(word)
            similarity.append(wv.n_similarity(t1_words,t2_words))

Generating a dataframe for the similarity scores for every pair of texts present in each row of the given dataset:

In [31]:
similarity = pd.DataFrame(similarity)
similarity.head()

,0
0,0.738640
1,0.667202
2,0.775820
3,0.658866
4,0.865341


In [32]:
df = pd.concat([df,similarity],axis=1)
df.head()

,text1,text2,0
0,broadband challeng tv view number european bro...,garden win doubl glasgow britain jason garden ...,0.738640
1,rap bos arrest drug find rap mogul marion suge...,amnesti chief lament war failur lack public ou...,0.667202
2,player burnout worri robinson england coach an...,hank greet wintri premier hollywood star tom h...,0.775820
3,heart oak cotonsport heart oak set ghanaian co...,redford vision sundanc despit sport corduroy c...,0.658866
4,sir paul rock super bowl crowd sir paul mccart...,mauresmo open victori la ameli mauresmo maria ...,0.865341


In [33]:
df.columns = ['text1','text2','Similarity Score']

In [34]:
df.head()

,text1,text2,Similarity Score
0,broadband challeng tv view number european bro...,garden win doubl glasgow britain jason garden ...,0.738640
1,rap bos arrest drug find rap mogul marion suge...,amnesti chief lament war failur lack public ou...,0.667202
2,player burnout worri robinson england coach an...,hank greet wintri premier hollywood star tom h...,0.775820
3,heart oak cotonsport heart oak set ghanaian co...,redford vision sundanc despit sport corduroy c...,0.658866
4,sir paul rock super bowl crowd sir paul mccart...,mauresmo open victori la ameli mauresmo maria ...,0.865341


In [35]:
df.to_csv('similarity_scores.csv')

## Saving and loading the cosine similarity model within the joblib library in order to use it during the time of model deployment:

In [38]:
joblib.dump(cosine_similarity,'model.pkl')

['model.pkl']

In [39]:
model = joblib.load('model.pkl')